In [1]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box,MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
from gymnasium import spaces
from gymnasium import ObservationWrapper
from stable_baselines3.common.monitor import Monitor

In [3]:
import pygame


C:\Users\Brian\anaconda3\envs\breakout\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [4]:
class Agent:
    def __init__(self, start_pos):
        self._pos = np.array(start_pos, dtype=np.int32)
        self._prev_state = None
        self._prev_prev_state = None
        self._carrying = None
        self._prev_goal_dists = []    # for stuck detection
    
    # def set_pos(self, new_pos):
    #     self.prev_prev_state = self.prev_state.copy() if self.prev_state is not None else None
    #     self.prev_state = self.state.copy()
    #     self.pos = np.array(new_pos, dtype=np.int32)

    # def set_carrying(self, package):
    #     self.carrying = package


In [5]:
class package:
    def __init__(self, pos):
        self._pos = pos
        self._delivered = False
        self._picked = False
        self._assigned = None
   


      

    def copy(self):
        return {
            "pos": self._pos,
            "picked": self._picked,
            "assigned_to": self._assigned,
            "delivered": self._delivered
        }

    def get_priority(self):
        return 0

In [6]:
class ExpressPackage(package):
    def get_priority(self):
        return 1

    

In [7]:
class HeavyPackage(package):
    def get_priority(self):
        return -1

    

In [8]:
def init_map():
    tok_map = [
        ['#'] * 28,
        ['#'] + ['.'] * 26 + ['#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#'] + ['.'] * 23 + ['.']  + ['.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#'] + ['.'] * 25 + ['O', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#'] + ['.'] * 23 + ['.']  + ['.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#', '.', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '@', '@', '@', '@', '.', '.', '.', '.', '.', '#'],
        ['#'] + ['.'] * 26 + ['#'],
        ['#'] * 28,
    ]

    return tok_map

In [9]:

def manhattan(a, b):
    """Return Manhattan (L1) distance between two 2-tuples/iterables (x,y)."""
    ax, ay = a
    bx, by = b
    return abs(ax - bx) + abs(ay - by)



In [1]:
# ##reg
# class WareHouseEnv(Env):
#     def __init__(self, start_pos=(4, 24),
#                  move_length=150,
#                  step_cost=-0.01,
#                  collision_penalty=-1.0,
#                  pickup_reward=+20.0,
#                  deliver_reward=+30.0,
#                  completed_reward=+60.0,
#                  max_num_packages=3):

#         # ---- ACTION & OBSERVATION SPACE ----
#         self.action_space = Discrete(4)
#         self.prev_state = None
#         self.prev_prev_state = None
#         self.prev_goal_dists = []



#         # observation = [ax, ay, gx, gy, carrying, blk_up, blk_down, blk_left, blk_right]
#         low  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32)
#         high = np.array([14, 27, 14, 27, 1, 1, 1, 1, 1], dtype=np.float32)


#         self.observation_space = Box(low=low, high=high, dtype=np.float32)

#         # ---- MAP SETUP ----
#         self.map = init_map()
#         self.obstacles = {'#', '@', 'O', '%'}

#         # ---- REWARDS ----
#         self.step_cost = step_cost
#         self.collision_penalty = collision_penalty
#         self.pickup_reward = pickup_reward
#         self.deliver_reward = deliver_reward
#         self.completed_reward = completed_reward

#         # ---- AGENT STATE ----
#         self.start_pos = np.array(start_pos, dtype=np.int32)
#         self.state = self.start_pos.copy()

#         self.max_move_length = move_length
#         self.move_length = move_length

#         self.packages = []
#         self.max_num_packages = max_num_packages
#         self.carrying = None

#         # ---- LOCATIONS ----
#         self.offload_positions = self._find_tiles('O')
#         self.shelf_pos = self._find_tiles('@')

#         # ---- RNG ----
#         self.np_random = np.random.RandomState()

#         # Spawn initial packages
#         for _ in range(3):
#             self.spawn_goals()

#     def init_pygame(self):
#         pygame.init()
#         self.tile_size = 24
#         self.colors = {
#             "empty": (240, 240, 240),
#             "wall":  (80, 80, 80),
#             "shelf": (139, 69, 19),
#             "offload": (0, 120, 255),
#             "package": (255, 165, 0),
#             "agent": (0, 200, 0)
#         }
#         self.screen = pygame.display.set_mode(
#             (self.tile_size * 28, self.tile_size * 15)
#         )
#         pygame.display.set_caption("Warehouse Environment")

#     # -----------------------------------------------------

#     def _find_tiles(self, tile_char):
#         out = []
#         for i, row in enumerate(self.map):
#             for j, c in enumerate(row):
#                 if c == tile_char:
#                     out.append((i, j))
#         return out

#     # -----------------------------------------------------

#     def _get_current_goal(self):
#         """Return nearest package OR offload point."""
#         ax, ay = self.state

#         if self.carrying is None and self.packages:
#             # nearest package
#             goal = min([p["pos"] for p in self.packages],
#                        key=lambda p: abs(p[0] - ax) + abs(p[1] - ay))
#         elif self.carrying is not None:
#             # deliver target
#             goal = self.offload_positions[0]
#         else:
#             # no goals
#             goal = (ax, ay)

#         return goal

#     # -----------------------------------------------------

#     def _shape_reward(self, prev_state, reward):
#         """Dense shaping based on distance to current goal."""
#         goal = self._get_current_goal()
#         gx, gy = goal

#         ax, ay = self.state
#         px, py = prev_state

#         old_dist = abs(px - gx) + abs(py - gy)
#         new_dist = abs(ax - gx) + abs(ay - gy)

#         # Moving toward goal → positive reward
#         reward += 0.01 * (old_dist - new_dist)

#         return reward

#     # -----------------------------------------------------

#     def spawn_goals(self):
#         if len(self.packages) < self.max_num_packages:
#             pos = self.shelf_pos[self.np_random.randint(len(self.shelf_pos))]
#             self.packages.append({"pos": pos, "picked": False})

#     # -----------------------------------------------------

#     def reset(self, seed=None, options=None):
#         if seed is None:
#             self.state = np.array(self.start_pos) + np.random.randint(-1, 2, size=2)
#         else:
#             self.state = self.start_pos.copy()

#         super().reset(seed=seed)
#         if seed is not None:
#             self.np_random.seed(seed)

#         self.state = self.start_pos.copy()
#         self.move_length = self.max_move_length

#         self.packages = []
#         self.carrying = None

#         for _ in range(3):
#             self.spawn_goals()

#         return self._obs(), {"packages": [p.copy() for p in self.packages]}

#     # -----------------------------------------------------

#     def _obs(self):
#         ax, ay = self.state
#         gx, gy = self._get_current_goal()
#         carrying_flag = 1 if self.carrying is not None else 0
    
#         # ----- blocked flags -----
#         blocked_up = 1 if (ax - 1 < 0 or self.map[ax - 1][ay] in self.obstacles) else 0
#         blocked_down = 1 if (ax + 1 >= 15 or self.map[ax + 1][ay] in self.obstacles) else 0
#         blocked_left = 1 if (ay - 1 < 0 or self.map[ax][ay - 1] in self.obstacles) else 0
#         blocked_right = 1 if (ay + 1 >= 28 or self.map[ax][ay + 1] in self.obstacles) else 0
    
#         return np.array([
#             ax, ay,
#             gx, gy,
#             carrying_flag,
#             blocked_up, blocked_down, blocked_left, blocked_right
#         ], dtype=np.float32)


#     # -----------------------------------------------------

#     def step(self, action):
#         ax, ay = self.state
#         prev_state = self.state.copy()
        
#         # -----------------------
#         # Movement proposal
#         # -----------------------
#         if action == 0:       # down
#             nx, ny = ax + 1, ay
#         elif action == 1:     # up
#             nx, ny = ax - 1, ay
#         elif action == 2:     # right
#             nx, ny = ax, ay + 1
#         elif action == 3:     # left
#             nx, ny = ax, ay - 1
        
#         reward = 0.0
#         blocked = False
        
#         # -----------------------
#         # Collision check
#         # -----------------------
#         if not (0 <= nx < 15 and 0 <= ny < 28) or self.map[nx][ny] in self.obstacles:
#             reward += self.collision_penalty
#             nx, ny = ax, ay        # revert
#             blocked = True
        
#         # -----------------------
#         # Step cost
#         # -----------------------
#         reward += self.step_cost
#         self.move_length -= 1
        
#         # -----------------------
#         # Pickup
#         # -----------------------
#         if self.carrying is None:
#             for idx, pkg in enumerate(self.packages):
#                 px, py = pkg["pos"]
#                 if abs(nx - px) + abs(ny - py) <= 1:
#                     self.carrying = idx
#                     self.packages.pop(idx)
#                     reward += self.pickup_reward
#                     break
        
#         # -----------------------
#         # Delivery
#         # -----------------------
#         if self.carrying is not None:
#             ox, oy = self.offload_positions[0]
#             if abs(nx - ox) + abs(ny - oy) <= 1:
#                 self.carrying = None
#                 reward += self.deliver_reward
        
#         # -----------------------
#         # Reward shaping
#         # -----------------------
#         temp_state = np.array([nx, ny])
#         old_state_saved = self.state
#         self.state = temp_state
#         reward = self._shape_reward(prev_state, reward)
#         self.state = old_state_saved
        
#         # -----------------------
#         # Reversal detection
#         # -----------------------
#         if self.prev_prev_state is not None:
#             reversed_move = (nx == self.prev_prev_state[0] and ny == self.prev_prev_state[1])
#             if reversed_move:
#                 gx, gy = self._get_current_goal()
#                 prev_prev_dist = abs(self.prev_prev_state[0] - gx) + abs(self.prev_prev_state[1] - gy)
#                 new_dist = abs(nx - gx) + abs(ny - gy)
#                 # Penalize only if reversal does NOT improve distance
#                 if new_dist >= prev_prev_dist:
#                     reward -= 0.05  # small penalty for bad reversal

#         gx, gy = self._get_current_goal()
#         current_dist = abs(nx - gx) + abs(ny - gy)
        
#         self.prev_goal_dists.append(current_dist)
#         if len(self.prev_goal_dists) > 10:   # keep buffer small
#             self.prev_goal_dists.pop(0)
        
#         # If last 5 distances are identical -> stuck
#         if len(self.prev_goal_dists) >= 5:
#             if len(set(self.prev_goal_dists[-5:])) == 1:
#                 reward -= 0.1  # small penalty for being stuck
        
#         # -----------------------
#         # No-movement penalty
#         # -----------------------
#         if blocked or (nx == ax and ny == ay):
#             reward -= 0.1
        
#         # -----------------------
#         # Final state update
#         # -----------------------
#         self.prev_prev_state = self.prev_state.copy() if self.prev_state is not None else None
#         self.prev_state = self.state.copy()
#         self.state = np.array([nx, ny], dtype=np.int32)
        
#         # -----------------------
#         # Termination conditions
#         # -----------------------
#         terminated = False
#         truncated = False
        
#         if self.carrying is None and len(self.packages) == 0:
#             reward += self.completed_reward
#             terminated = True
        
#         if self.move_length <= 0:
#             truncated = True
        
#         return self._obs(), reward, terminated, truncated, {}



#     # -----------------------------------------------------

#     def render(self):
#         for event in pygame.event.get():
#             if event.type == pygame.QUIT:
#                 pygame.quit()
    
#         for i in range(15):
#             for j in range(28):
#                 tile = self.map[i][j]
    
#                 if tile in {"#", "%"}:
#                     color = self.colors["wall"]
#                 elif tile == "@":
#                     color = self.colors["shelf"]
#                 elif tile == "O":
#                     color = self.colors["offload"]
#                 else:
#                     color = self.colors["empty"]
    
#                 pygame.draw.rect(
#                     self.screen,
#                     color,
#                     (j * self.tile_size, i * self.tile_size,
#                      self.tile_size, self.tile_size)
#                 )
    
#         # Draw packages
#         for p in self.packages:
#             x, y = p["pos"]
#             pygame.draw.rect(
#                 self.screen,
#                 self.colors["package"],
#                 (y * self.tile_size, x * self.tile_size,
#                  self.tile_size, self.tile_size)
#             )
    
#         # Draw agent
#         ax, ay = self.state
#         pygame.draw.rect(
#             self.screen,
#             self.colors["agent"],
#             (ay * self.tile_size, ax * self.tile_size,
#              self.tile_size, self.tile_size)
#         )
    
#         pygame.display.flip()
#         pygame.time.wait(40)  # ~25 FPS


In [7]:
## env with agent class
# class WareHouseEnv(Env):
#     def __init__(self, start_pos=(4, 24),
#                  move_length=150,
#                  step_cost=-0.01,
#                  collision_penalty=-1.0,
#                  pickup_reward=+20.0,
#                  deliver_reward=+30.0,
#                  completed_reward=+60.0,
#                  max_num_packages=3):

#         # ---- ACTION & OBSERVATION SPACE ----
#         self.action_space = Discrete(4)
#         self.agent = Agent(start_pos)




#         # observation = [ax, ay, gx, gy, carrying, blk_up, blk_down, blk_left, blk_right]
#         low  = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32)
#         high = np.array([14, 27, 14, 27, 1, 1, 1, 1, 1], dtype=np.float32)


#         self.observation_space = Box(low=low, high=high, dtype=np.float32)

#         # ---- MAP SETUP ----
#         self.map = init_map()
#         self.obstacles = {'#', '@', 'O', '%'}

#         # ---- REWARDS ----
#         self.step_cost = step_cost
#         self.collision_penalty = collision_penalty
#         self.pickup_reward = pickup_reward
#         self.deliver_reward = deliver_reward
#         self.completed_reward = completed_reward

#         # ---- AGENT STATE ----
#         self.start_pos = np.array(start_pos, dtype=np.int32)
    

#         self.max_move_length = move_length
#         self.move_length = move_length

#         self.packages = []
#         self.max_num_packages = max_num_packages
        

#         # ---- LOCATIONS ----
#         self.offload_positions = self._find_tiles('O')
#         self.shelf_pos = self._find_tiles('@')

#         # ---- RNG ----
#         self.np_random = np.random.RandomState()

#         # Spawn initial packages
#         for _ in range(3):
#             self.spawn_goals()

#     def init_pygame(self):
#         pygame.init()
#         self.tile_size = 24
#         self.colors = {
#             "empty": (240, 240, 240),
#             "wall":  (80, 80, 80),
#             "shelf": (139, 69, 19),
#             "offload": (0, 120, 255),
#             "package": (255, 165, 0),
#             "agent": (0, 200, 0)
#         }
#         self.screen = pygame.display.set_mode(
#             (self.tile_size * 28, self.tile_size * 15)
#         )
#         pygame.display.set_caption("Warehouse Environment")

#     # -----------------------------------------------------

#     def _find_tiles(self, tile_char):
#         out = []
#         for i, row in enumerate(self.map):
#             for j, c in enumerate(row):
#                 if c == tile_char:
#                     out.append((i, j))
#         return out

#     # -----------------------------------------------------

#     def _get_current_goal(self):
#         """Return nearest package OR offload point."""
#         ax, ay = self.agent.pos

#         if self.agent.carrying is None and self.packages:
#             # nearest package
#             goal = min([p["pos"] for p in self.packages],
#                        key=lambda p: abs(p[0] - ax) + abs(p[1] - ay))
#         elif self.agent.carrying is not None:
#             # deliver target
#             goal = self.offload_positions[0]
#         else:
#             # no goals
#             goal = (ax, ay)

#         return goal

#     # -----------------------------------------------------

#     def _shape_reward(self, prev_state, reward):
#         """Dense shaping based on distance to current goal."""
#         goal = self._get_current_goal()
#         gx, gy = goal

#         ax, ay = self.agent.pos
#         px, py = self.agent.prev_state

#         old_dist = abs(px - gx) + abs(py - gy)
#         new_dist = abs(ax - gx) + abs(ay - gy)

#         # Moving toward goal → positive reward
#         reward += 0.01 * (old_dist - new_dist)

#         return reward

#     # -----------------------------------------------------

#     def spawn_goals(self):
#         if len(self.packages) < self.max_num_packages:
#             pos = self.shelf_pos[self.np_random.randint(len(self.shelf_pos))]
#             self.packages.append({"pos": pos, "picked": False})

#     # -----------------------------------------------------

#     def reset(self, seed=None, options=None):
#         if seed is None:
#             self.agent.pos = np.array(self.start_pos) + np.random.randint(-1, 2, size=2)
#         else:
#             self.agent.pos = self.start_pos.copy()

#         super().reset(seed=seed)
#         if seed is not None:
#             self.np_random.seed(seed)

#         # self.agent = Agent(start_pos)
#         self.move_length = self.max_move_length

#         self.packages = []
#         self.agent.carrying = None

#         for _ in range(3):
#             self.spawn_goals()

#         return self._obs(), {"packages": [p.copy() for p in self.packages]}

#     # -----------------------------------------------------

#     def _obs(self):
#         ax, ay = self.agent.pos
#         gx, gy = self._get_current_goal()
#         carrying_flag = 1 if self.agent.carrying is not None else 0
    
#         # ----- blocked flags -----
#         blocked_up = 1 if (ax - 1 < 0 or self.map[ax - 1][ay] in self.obstacles) else 0
#         blocked_down = 1 if (ax + 1 >= 15 or self.map[ax + 1][ay] in self.obstacles) else 0
#         blocked_left = 1 if (ay - 1 < 0 or self.map[ax][ay - 1] in self.obstacles) else 0
#         blocked_right = 1 if (ay + 1 >= 28 or self.map[ax][ay + 1] in self.obstacles) else 0
    
#         return np.array([
#             ax, ay,
#             gx, gy,
#             carrying_flag,
#             blocked_up, blocked_down, blocked_left, blocked_right
#         ], dtype=np.float32)


#     # -----------------------------------------------------

#     def step(self, action):
#         ax, ay = self.agent.pos
#         self.agent.prev_state = self.agent.pos.copy()
        
#         # -----------------------
#         # Movement proposal
#         # -----------------------
#         if action == 0:       # down
#             nx, ny = ax + 1, ay
#         elif action == 1:     # up
#             nx, ny = ax - 1, ay
#         elif action == 2:     # right
#             nx, ny = ax, ay + 1
#         elif action == 3:     # left
#             nx, ny = ax, ay - 1
        
#         reward = 0.0
#         blocked = False
        
#         # -----------------------
#         # Collision check
#         # -----------------------
#         if not (0 <= nx < 15 and 0 <= ny < 28) or self.map[nx][ny] in self.obstacles:
#             reward += self.collision_penalty
#             nx, ny = ax, ay        # revert
#             blocked = True
        
#         # -----------------------
#         # Step cost
#         # -----------------------
#         reward += self.step_cost
#         self.move_length -= 1
        
#         # -----------------------
#         # Pickup
#         # -----------------------
#         if self.agent.carrying is None:
#             for idx, pkg in enumerate(self.packages):
#                 px, py = pkg["pos"]
#                 if abs(nx - px) + abs(ny - py) <= 1:
#                     self.agent.carrying = idx
#                     self.packages.pop(idx)
#                     reward += self.pickup_reward
#                     break
        
#         # -----------------------
#         # Delivery
#         # -----------------------
#         if self.agent.carrying is not None:
#             ox, oy = self.offload_positions[0]
#             if abs(nx - ox) + abs(ny - oy) <= 1:
#                 self.agent.carrying = None
#                 reward += self.deliver_reward
        
#         # -----------------------
#         # Reward shaping
#         # -----------------------
#         temp_state = np.array([nx, ny])
#         old_state_saved = self.agent.pos.copy()
#         self.agent.pos = temp_state
#         reward = self._shape_reward(self.agent.prev_state.copy(), reward)
#         self.agent.pos = old_state_saved
        
#         # -----------------------
#         # Reversal detection
#         # -----------------------
#         if self.agent.prev_prev_state is not None:
#             reversed_move = (nx == self.agent.prev_prev_state[0] and ny == self.agent.prev_prev_state[1])
#             if reversed_move:
#                 gx, gy = self._get_current_goal()
#                 prev_prev_dist = abs(self.agent.prev_prev_state[0] - gx) + abs(self.agent.prev_prev_state[1] - gy)
#                 new_dist = abs(nx - gx) + abs(ny - gy)
#                 # Penalize only if reversal does NOT improve distance
#                 if new_dist >= prev_prev_dist:
#                     reward -= 0.05  # small penalty for bad reversal

#         gx, gy = self._get_current_goal()
#         current_dist = abs(nx - gx) + abs(ny - gy)
        
#         self.agent.prev_goal_dists.append(current_dist)
#         if len(self.agent.prev_goal_dists) > 10:   # keep buffer small
#             self.agent.prev_goal_dists.pop(0)
        
#         # If last 5 distances are identical -> stuck
#         if len(self.agent.prev_goal_dists) >= 5:
#             if len(set(self.agent.prev_goal_dists[-5:])) == 1:
#                 reward -= 0.1  # small penalty for being stuck
        
#         # -----------------------
#         # No-movement penalty
#         # -----------------------
#         if blocked or (nx == ax and ny == ay):
#             reward -= 0.1
        
#         # -----------------------
#         # Final state update
#         # -----------------------
#         self.agent.prev_prev_state = self.agent.prev_state.copy() if self.agent.prev_state is not None else None
#         self.agent.prev_state = self.agent.pos.copy()
#         self.agent.pos = np.array([nx, ny], dtype=np.int32)
        
#         # -----------------------
#         # Termination conditions
#         # -----------------------
#         terminated = False
#         truncated = False
        
#         if self.agent.carrying is None and len(self.packages) == 0:
#             reward += self.completed_reward
#             terminated = True
        
#         if self.move_length <= 0:
#             truncated = True
        
#         return self._obs(), reward, terminated, truncated, {}



#     # -----------------------------------------------------

#     def render(self):
#         for event in pygame.event.get():
#             if event.type == pygame.QUIT:
#                 pygame.quit()
#                 sys.exit()
    
#         for i in range(15):
#             for j in range(28):
#                 tile = self.map[i][j]
    
#                 if tile in {"#", "%"}:
#                     color = self.colors["wall"]
#                 elif tile == "@":
#                     color = self.colors["shelf"]
#                 elif tile == "O":
#                     color = self.colors["offload"]
#                 else:
#                     color = self.colors["empty"]
    
#                 pygame.draw.rect(
#                     self.screen,
#                     color,
#                     (j * self.tile_size, i * self.tile_size,
#                      self.tile_size, self.tile_size)
#                 )
    
#         # Draw packages
#         for p in self.packages:
#             x, y = p["pos"]
#             pygame.draw.rect(
#                 self.screen,
#                 self.colors["package"],
#                 (y * self.tile_size, x * self.tile_size,
#                  self.tile_size, self.tile_size)
#             )
    
#         # Draw agent
#         ax, ay = self.agent.pos
#         pygame.draw.rect(
#             self.screen,
#             self.colors["agent"],
#             (ay * self.tile_size, ax * self.tile_size,
#              self.tile_size, self.tile_size)
#         )
    
#         pygame.display.flip()
#         pygame.time.wait(40)  # ~25 FPS

In [10]:
## class with two agent
class WareHouseEnv(Env):
    def __init__(
        self,
        start_pos=(4, 24),
        move_length=200,
        step_cost=-0.01,
        collision_penalty=-1.0,
        pickup_reward=+20.0,
        deliver_reward=+30.0,
        completed_reward=+60.0,
        max_num_packages=4,
        agent_num=2,
        num_of_express_pkg = 1,
        num_of_heavy_pkg=1,
    ):
        self.agent_num = agent_num
        self.start_pos = np.array(start_pos, dtype=np.int32)

        self.agents = [Agent(self.start_pos.copy()  + np.random.randint(-1, 2, size=2)) for _ in range(agent_num)]

        # ACTION SPACE
        self.action_space = MultiDiscrete([4] * self.agent_num)

        # OBS SPACE
        obs_per_agent = 11
        low_one = np.array([
            0, 0,     # ax, ay
            0, 0,     # gx, gy
            0,        # carrying_flag
            0, 0, 0, 0,  # blocked flags
            -14, -27
        ], dtype=np.float32)

        high_one = np.array([
            14, 27,
            14, 27,
            1,
            1, 1, 1, 1,
            14, 27
        ], dtype=np.float32)

        low = np.tile(low_one, self.agent_num)
        high = np.tile(high_one, self.agent_num)
        self.observation_space = Box(low=low, high=high, dtype=np.float32)

        # MAP
        self.map = init_map()
        self.obstacles = {"#", "@", "O", "%"}

        # Rewards
        self.step_cost = step_cost
        self.collision_penalty = collision_penalty
        self.pickup_reward = pickup_reward
        self.deliver_reward = deliver_reward
        self.completed_reward = completed_reward

        # Agent / env state
        self.max_move_length = move_length
        self.move_length = move_length
        self.packages = []
        self.max_num_packages = max_num_packages
        self.num_of_express_pkg = num_of_express_pkg
        self.num_of_heavy_pkg = num_of_heavy_pkg
        # Locations
        self.offload_positions = self._find_tiles("O")
        self.shelf_pos = self._find_tiles("@")
        self.drop_zone = self.offload_positions[0]
     


        # RNG
        self.np_random = np.random.RandomState()

        # spawn initial packages
        for _ in range(max_num_packages - num_of_express_pkg - num_of_heavy_pkg):
            self.spawn_goals()
        for _ in range(num_of_express_pkg):
            self.spawn_goals("express")
        for _ in range(num_of_heavy_pkg):
            self.spawn_goals("heavy")

    # ------------------------------------------------------
    def init_pygame(self):
        pygame.init()
        self.tile_size = 24
        self.colors = {
            "empty": (240, 240, 240),
            "wall": (80, 80, 80),
            "shelf": (139, 69, 19),
            "offload": (0, 120, 255),
            "normal_package": (255, 165, 0),
            "express_package": (255,0,0),
            "heavy_package": (128,128,128),
            "agent": (0, 200, 0),
        }
        self.screen = pygame.display.set_mode(
            (self.tile_size * 28, self.tile_size * 15)
        )
        pygame.display.set_caption("Warehouse Environment")

    # ------------------------------------------------------
    def _find_tiles(self, tile_char):
        out = []
        for i, row in enumerate(self.map):
            for j, c in enumerate(row):
                if c == tile_char:
                    out.append((i, j))
        return out

    # ------------------------------------------------------
    def assign_packages(self):
        """
        Assign each free agent exactly one package.
        Deterministic tie-breaking:
          - higher priority first
          - if tie, closer distance first
          - stable agent ordering
        """

        ### [FIX STEP 2] stable free_agents ordering ###
        free_agents = []
        for i, agent in enumerate(self.agents):
            if agent._carrying is None:
                has_assigned = any(
                    (p._assigned == i and not p._picked and not p._delivered)
                    for p in self.packages
                )
                if not has_assigned:
                    free_agents.append(i)

        if not free_agents:
            return

        free_agents.sort()    # <-- DETERMINISTIC

        ### [FIX STEP 2] deterministic unassigned package list ###
        unassigned_pkgs = []
        for idx, p in enumerate(self.packages):
            if (not p._picked) and (not p._delivered) and (p._assigned is None):
                unassigned_pkgs.append(idx)

        if not unassigned_pkgs:
            return

        # For each agent in deterministic order, assign best package
        for a_idx in free_agents:
            ax, ay = self.agents[a_idx]._pos

            best = None  # (priority, -dist, p_idx)
            for p_idx in unassigned_pkgs:
                pkg = self.packages[p_idx]
                px, py = pkg._pos
                dist = abs(px - ax) + abs(py - ay)
                cand = (pkg.get_priority(), -dist, p_idx)

                if best is None or cand > best:
                    best = cand

            if best is None:
                continue

            _, _, best_idx = best
            self.packages[best_idx]._assigned = a_idx

            unassigned_pkgs.remove(best_idx)


    # ------------------------------------------------------
    def _get_current_goal(self, i):
        agent = self.agents[i]

        if agent._carrying is not None:
            return self.drop_zone
        
        for pkg in self.packages:
            if pkg._assigned == i:
                return pkg._pos

        return None

    # ------------------------------------------------------
    def _shape_reward(self, prev_state, reward, agent_index):
        goal = self._get_current_goal(agent_index)
        if goal is None:
            return reward

        gx, gy = goal
        ax, ay = self.agents[agent_index]._pos
        px, py = prev_state

        old_dist = abs(px - gx) + abs(py - gy)
        new_dist = abs(ax - gx) + abs(ay - gy)

        return reward + 0.01 * (old_dist - new_dist)

    # ------------------------------------------------------
    def spawn_goals(self, type=None):
        if len(self.packages) < self.max_num_packages:

            existing = {tuple(p._pos) for p in self.packages if not getattr(p, "_picked", False)}

            free_shelves = [
                s for s in self.shelf_pos
                if tuple(s) not in existing
            ]

            if len(free_shelves) == 0:
                return

            pos = free_shelves[self.np_random.randint(len(free_shelves))]

            if type == "express":
                new_pkg = ExpressPackage(pos)
            elif type == "heavy":
                new_pkg = HeavyPackage(pos)
            else:
                new_pkg = package(pos)

            ### [FIX STEP 3] unified initialization of all package flags ###
            new_pkg._picked = False
            new_pkg._assigned = None
            new_pkg._delivered = False

            # initialize optional internal fields to prevent hasattr-based divergence
            new_pkg._progress_steps = 0
            new_pkg._last_distance = None
            new_pkg._cooldown_agent = None
            new_pkg._cooldown_steps = 0
            ### [END FIX STEP 3] ###

            self.packages.append(new_pkg)


    


    # ------------------------------------------------------
    def reset(self, seed=None, options=None):
        for i in range(self.agent_num):
            agent = self.agents[i]
                
            if seed is None:
                agent._pos = self.start_pos + np.random.randint(-1, 2, size=2)
            else:
                agent._pos = self.start_pos.copy()

            agent._carrying = None
            agent._prev_state = agent._pos.copy()
            agent._prev_prev_state = None
            agent._prev_goal_dists = []

        super().reset(seed=seed)
        if seed is not None:
            self.np_random.seed(seed)
        

        self.move_length = self.max_move_length
        self.packages = []
        for _ in range(self.max_num_packages - self.num_of_express_pkg - self.num_of_heavy_pkg):
            self.spawn_goals()
        for _ in range(self.num_of_express_pkg):
            self.spawn_goals("express")
        for _ in range(self.num_of_heavy_pkg):
            self.spawn_goals("heavy")

        return self._obs(), {"packages": [p.copy() for p in self.packages]}

    # ------------------------------------------------------
    def _obs(self):
        obs_list = []

        for i in range(self.agent_num):
            agent = self.agents[i]
            ax, ay = agent._pos

            goal = self._get_current_goal(i)
            if goal is None:
                gx, gy = ax, ay
            else:
                gx, gy = goal

            carrying_flag = 1 if agent._carrying is not None else 0

            blocked_up = 1 if (ax - 1 < 0 or self.map[ax - 1][ay] in self.obstacles) else 0
            blocked_down = 1 if (ax + 1 >= 15 or self.map[ax + 1][ay] in self.obstacles) else 0
            blocked_left = 1 if (ay - 1 < 0 or self.map[ax][ay - 1] in self.obstacles) else 0
            blocked_right = 1 if (ay + 1 >= 28 or self.map[ax][ay + 1] in self.obstacles) else 0

            other_rel = []
            for j, other in enumerate(self.agents):
                if i == j:
                    continue
                ox, oy = other._pos
                other_rel.extend([ox - ax, oy - ay])

            obs_list.extend([
                ax, ay,
                gx, gy,
                carrying_flag,
                blocked_up, blocked_down, blocked_left, blocked_right,
                *other_rel
            ])

        return np.array(obs_list, dtype=np.float32)

    # ------------------------------------------------------
    def step(self, actions):
        rewards = [0.0 for _ in range(self.agent_num)]
    
        # always update package assignments
        self.assign_packages()
    
        # Update agent history once (prev_prev <- prev <- current)
        for agent in self.agents:
            agent._prev_prev_state = None if agent._prev_state is None else agent._prev_state.copy()
            agent._prev_state = agent._pos.copy()
        
        
    
        # -------------------------
        # PHASE 1: propose movement
        # -------------------------
        proposed_positions = []
        blocked_flags = [False] * self.agent_num
    
        for i in range(self.agent_num):
            agent = self.agents[i]
            ax, ay = agent._pos
    
            a = actions[i]
    
            # === PATCHED: explicit, correct mapping (0=UP,1=DOWN,2=RIGHT,3=LEFT) ===
            if a == 0:
                nx, ny = ax - 1, ay
            elif a == 1:
                nx, ny = ax + 1, ay
            elif a == 2:
                nx, ny = ax, ay + 1
            elif a == 3:
                nx, ny = ax, ay - 1
            else:
                nx, ny = ax, ay
    
            blocked = False
    
            if not (0 <= nx < 15 and 0 <= ny < 28) or self.map[nx][ny] in self.obstacles:
                rewards[i] += self.collision_penalty
                nx, ny = ax, ay
                blocked = True
    
            other_positions = [tuple(a._pos) for j, a in enumerate(self.agents) if j != i]
            if (nx, ny) in other_positions:
                rewards[i] += self.collision_penalty * 2.0
                nx, ny = ax, ay
                blocked = True
    
            rewards[i] += self.step_cost
    
            proposed_positions.append((nx, ny))
            blocked_flags[i] = blocked
    
        # --- FIX SWAP COLLISION ---
        swap_block = [False] * self.agent_num
        for i in range(self.agent_num):
            for j in range(i + 1, self.agent_num):
                if (tuple(self.agents[i]._pos) == proposed_positions[j] and
                    tuple(self.agents[j]._pos) == proposed_positions[i]):
                    swap_block[i] = True
                    swap_block[j] = True
    
        for i in range(self.agent_num):
            if swap_block[i]:
                # rewards[i] += self.collision_penalty * 2.0
                
                proposed_positions[i] = tuple(self.agents[i]._pos)
                blocked_flags[i] = True
    
        # -------------------------
        # PHASE 2: update positions
        # -------------------------
        for i in range(self.agent_num):
            old_pos = self.agents[i]._pos.copy()
            self.agents[i]._pos = np.array(proposed_positions[i])
    
        # -------------------------
        # PHASE 3: package pickup
        # -------------------------
        for i in range(self.agent_num):
            agent = self.agents[i]
            if agent._carrying is not None:
                continue
    
            ax, ay = agent._pos
            closest_idx = None
            closest_dist = float("inf")
    
            for idx, pkg in enumerate(self.packages):
                # if not hasattr(pkg, "_picked"):
                #     pkg._picked = False
                # if not hasattr(pkg, "_delivered"):
                #     pkg._delivered = False
                # if not hasattr(pkg, "_assigned"):
                #     pkg._assigned = None
    
                if pkg._picked or pkg._delivered:
                    continue
    
                px, py = pkg._pos
                d = abs(px - ax) + abs(py - ay)
    
                if d <= 1 and d < closest_dist:
                    closest_dist = d
                    closest_idx = idx
    
            if closest_idx is not None:
                pkg = self.packages[closest_idx]
                pkg._picked = True
                pkg._assigned = None
                agent._carrying = pkg
                rewards[i] += self.pickup_reward
    
        # -------------------------
        # PHASE 4: delivery
        # -------------------------
        for i in range(self.agent_num):
            agent = self.agents[i]
            if agent._carrying is None:
                continue
    
            ox, oy = self.offload_positions[0]
            if abs(agent._pos[0] - ox) + abs(agent._pos[1] - oy) <= 1:
                try:
                    agent._carrying._delivered = True
                except:
                    pass
    
                agent._carrying = None
                rewards[i] += self.deliver_reward
    
        # -------------------------
        # PHASE 5: shaping
        # -------------------------
        for i in range(self.agent_num):
            agent = self.agents[i]
    
            rewards[i] = self._shape_reward(agent._prev_state.copy(), rewards[i], i)
    
            if agent._prev_prev_state is not None:
                reversed_move = (
                    agent._pos[0] == agent._prev_prev_state[0] and
                    agent._pos[1] == agent._prev_prev_state[1]
                )
                if reversed_move:
                    goal = self._get_current_goal(i)
                    if goal is not None:
                        gx, gy = goal
                        prev_prev_dist = abs(agent._prev_prev_state[0] - gx) + abs(agent._prev_prev_state[1] - gy)
                        new_dist = abs(agent._pos[0] - gx) + abs(agent._pos[1] - gy)
                        if new_dist >= prev_prev_dist:
                            rewards[i] -= 0.05
    
            goal = self._get_current_goal(i)
            if goal is not None:
                gx, gy = goal
                current_dist = abs(agent._pos[0] - gx) + abs(agent._pos[1] - gy)
            else:
                current_dist = 0
    
            agent._prev_goal_dists.append(current_dist)
            if len(agent._prev_goal_dists) > 10:
                agent._prev_goal_dists.pop(0)
    
            if len(agent._prev_goal_dists) >= 5 and len(set(agent._prev_goal_dists[-5:])) == 1:
                rewards[i] -= 0.1
    
            if blocked_flags[i] or (goal is not None and np.array_equal(agent._pos, agent._prev_state)):
                rewards[i] -= 0.1
    
        # -------------------------
        # termination logic
        # -------------------------
        self.move_length -= 1
    
        all_delivered = (len(self.packages) == 0) or all(getattr(p, "_delivered", False) for p in self.packages)
        terminated = all_delivered and all(a._carrying is None for a in self.agents)
        truncated = self.move_length <= 0
    
        total_reward = sum(rewards)
        return self._obs(), total_reward, terminated, truncated, {}



    # ------------------------------------------------------
    def render(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        for i in range(15):
            for j in range(28):
                tile = self.map[i][j]

                if tile in {"#", "%"}:
                    color = self.colors["wall"]
                elif tile == "@":
                    color = self.colors["shelf"]
                elif tile == "O":
                    color = self.colors["offload"]
                else:
                    color = self.colors["empty"]

                pygame.draw.rect(
                    self.screen, color,
                    (j * self.tile_size, i * self.tile_size,
                     self.tile_size, self.tile_size)
                )

        for p in self.packages:
            if getattr(p, "_picked", False):
                continue  # skip picked packages
            if isinstance(p, ExpressPackage):
                pkg_color = self.colors["express_package"]
            elif isinstance(p, HeavyPackage):
                pkg_color = self.colors["heavy_package"]
            else:
                pkg_color = self.colors["normal_package"]
            x, y = p._pos
            pygame.draw.rect(
                self.screen, pkg_color,
                (y * self.tile_size, x * self.tile_size,
                 self.tile_size, self.tile_size)
            )


        for i in range(self.agent_num):
            ax, ay = self.agents[i]._pos
            pygame.draw.rect(
                self.screen, self.colors["agent"],
                (ay * self.tile_size, ax * self.tile_size,
                 self.tile_size, self.tile_size)
            )

        pygame.display.flip()
        pygame.time.wait(40)

In [11]:
env = WareHouseEnv()

obs, info = env.reset()

print("Initial observation shape:", obs.shape)
print("Initial packages:", info["packages"])


Initial observation shape: (22,)
Initial packages: [{'pos': (6, 8), 'picked': False, 'assigned_to': None, 'delivered': False}, {'pos': (9, 16), 'picked': False, 'assigned_to': None, 'delivered': False}, {'pos': (9, 15), 'picked': False, 'assigned_to': None, 'delivered': False}, {'pos': (12, 16), 'picked': False, 'assigned_to': None, 'delivered': False}]


In [12]:
env = WareHouseEnv()
env.init_pygame()

env = Monitor(env)

In [126]:
import time
import os
episodes = 1
for episode in range(1, episodes+1):
    obs, info = env.reset()
    done = False
    score = 0

    while not done:
        # Clear console (optional, makes the render look like animation)
        os.system("cls" if os.name == "nt" else "clear")

        env.render()       # <-- render current state

        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)

        done = terminated or truncated
        score += reward

        time.sleep(0.05)   # slow down for visibility (adjust as needed)

    print(f"Episode {episode} Score {score}")

env.close()

KeyboardInterrupt: 

In [13]:
pygame.quit()

In [14]:
log_path = os.path.join("Training","Logs")
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)


Using cpu device
Wrapping the env in a DummyVecEnv.


In [15]:
model.learn(total_timesteps=2000000)

Logging to Training\Logs\PPO_76
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -87.8    |
| time/              |          |
|    fps             | 2424     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -90.1       |
| time/                   |             |
|    fps                  | 1383        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011774552 |
|    clip_fraction        | 0.164       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.76       |
|    explained_variance   | 0.0447      

In [86]:
##load model 
model = PPO.load(r"C:\Users\Brian\Downloads\warehouse\Q_learn\Training\Saved Models\WareHouse_Model_multi_ver3_PPO.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [109]:
env = WareHouseEnv()
env.init_pygame()

In [110]:
import time
import os

episodes = 5
for episode in range(1, episodes+1):
    obs, info = env.reset()
    done = False
    score = 0

    while not done:
        # Clear console (optional, makes the render look like animation)
        os.system("cls" if os.name == "nt" else "clear")

        env.render()       # <-- render current state

        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)

        done = terminated or truncated
        score += reward

        time.sleep(0.05)   # slow down for visibility (adjust as needed)

    print(f"Episode {episode} Score {score}")

env.close()


Episode 1 Score -23.17000000000006


KeyboardInterrupt: 

In [17]:
warehouse_path = os.path.join('Training', 'Saved Models', 'WareHouse_Model_multi_ver4_PPO')

In [18]:
model.save(warehouse_path)